In [ ]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

In [ ]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [ ]:
itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf_recommender = ItemKNNCFRecommender(data_train, verbose=False)
itemknncf_recommender.fit(**itemknncf_study.best_params)

In [ ]:
p3beta_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3beta_recommender = RP3betaRecommender(data_train, verbose=False)
p3beta_recommender.fit(**p3beta_study.best_params)

In [ ]:
study = optuna.create_study(
    study_name="ItemKNNCF + P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    
    recommender = ItemKNNCFRecommender(data_train, verbose=False)

    recommender.W_sparse = alpha * itemknncf_recommender.W_sparse + (1 - alpha) * p3beta_recommender.W_sparse
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=300)